# Day 7

## Part 1
- commands are prepended with `$`
- `cd` is cd
- `cd /` goes back to the top directory
- `ls` is ls
    - files are printed as: `<size> <name>`
    - directories are printed as: `dir <name>`

`Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?`

- Directory sizes are counted independently so the final total can contain the same file several times

In [6]:
from utils import parse_from_file, ParseConfig

parser = ParseConfig('\n$ ls\n', ParseConfig('\n', str))

terminal_output = parse_from_file('day_7.txt', parser)

print(terminal_output[:2])

[['$ cd /'], ['dir lhrfs', '193233 mvsjmrtn', 'dir nwh', 'dir pjsd', 'dir qfrrtb', '31987 zzdfcs', '$ cd lhrfs']]


In [ ]:
def get_file_system(terminal_output: list[list[str]]) -> dict:
    """
    maps out a file system from a terminal output
    """
    for block in terminal_output:
        